In [1]:
test_str = """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1"""

def parse_in(data_str=None):
    if not data_str:
        data_str = open("Inputs/Day12.txt").read().strip()
    
    ret_val = [((y:=l.split())[0], tuple(int(x) for x in y[1].split(","))) for l in data_str.split("\n")]
    return ret_val

parse_in(test_str)

[('???.###', (1, 1, 3)),
 ('.??..??...?##.', (1, 1, 3)),
 ('?#?#?#?#?#?#?#?', (1, 3, 1, 6)),
 ('????.#...#...', (4, 1, 1)),
 ('????.######..#####.', (1, 6, 5)),
 ('?###????????', (3, 2, 1))]

In [2]:
from itertools import combinations

def part1(data_in):
    total = 0
    for d in data_in:
        g_count = len(d[1])
        extra = len(d[0]) - sum(d[1]) - g_count + 1
        #nonograms solving technique, find all the possible combinations of the groupings and test them
        # one by one
        options = combinations(range(g_count + extra), g_count)
        o = list(options)
        springs = ["#"*i for i in d[1]]
        match_count = 0
        for x in o:
            diffs = [x[i + 1] - x[i] for i in range(len(x) - 1)]
            diffs = ["."*i for i in diffs]            
            
            val = [None]*(2*g_count - 1)
            val[::2] = springs
            val[1::2] = diffs
            val_str = "."*x[0] + "".join(val)
            val_str += "."*(len(d[0]) - len(val_str))
            
            for i in range(len(val_str)):
                if val_str[i] != d[0][i] and d[0][i] != "?":
                    break
            else:
                match_count += 1
        total += match_count
                
    return total

        
part1(parse_in()) #should be 8419

8419

In [3]:
from functools import cache

@cache
def check(puzzle_str, groups):
    #Each group needs a gap of 1 between them so there is a minimum valid "input length"
    if len(puzzle_str) < (sum(groups) + len(groups) - 1):
        #then there can be no more matches here
        return 0    
    
    if puzzle_str[0] == ".":
        #puzzle[0] is a . so remove all the . in front as they are not needed
        return check(puzzle_str.lstrip("."), groups)
    elif puzzle_str[0] == "?":
        #puzzle[0] is a ?, return the sum of the cases where it is either a . or #
        #      . case just trim the one dot    # case add the # in front
        return check(puzzle_str[1:], groups) + check("#" + puzzle_str[1:], groups)
    else:#puzzle_str[0] must be a "#"
        #there are enough "#" or "?" for the first group
        if all(x in "#?" for x in puzzle_str[0:groups[0]]):
            #If there was only one group left, check that no more groups are "required"
            if len(groups) == 1:
                #if there are no more "#" left return 1 else 0
                return all(x != "#" for x in puzzle_str[groups[0]:])            
            
            #Immediately after this "group" there is another #, this is a failure
            if puzzle_str[groups[0]] == "#":
                return 0
                
            return check(puzzle_str[groups[0]+1:], groups[1:])
        else:
            return 0


def part2(data_in):
    check.cache_clear()
    total = 0
    
    for d in data_in:
        puzzle_str = "?".join((d[0], d[0],d[0], d[0],d[0]))
        groups = d[1]*5
        #remove opening and trailing ".", no need to carry them around
        total += check(puzzle_str.strip(".").lstrip("."), groups)
    
    return total

print(part2(parse_in())) #should be 160500973317706


160500973317706


In [4]:
%timeit part1(parse_in())
%timeit part2(parse_in())

110 ms ± 688 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
200 ms ± 1.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
